Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Libaries

In [ ]:
import os, sys, json, random
import numpy as np
from numpy import dstack
import time
from matplotlib import pyplot as plt
import statistics

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn import metrics

from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

Randomness seed for reproducibility

In [ ]:
seed = 1
random.seed(seed)

Reading data

In [ ]:
benign_data    = {}
malware_data   = {}
report = {}

benign_dict = {}
malware_dict   = {}

benign_files = ["/content/drive/MyDrive/FYP/open_source_data/amd_desktop_metadata/benign_post_pca_data.txt",
                "/content/drive/MyDrive/FYP/open_source_data/amd_python_benign_metadata/python_benign_post_pca_data.txt"
                ]
malware_files = ["/content/drive/MyDrive/FYP/open_source_data/amd_malware_metadata/malware_post_pca_data.txt"]

print("Files loaded properly")
#--------------------------------------------------------------------------------------------------------------------------------#

# Benign data
for files in benign_files:

  with open(files, 'r') as data_file:
    json_data = json.load(data_file)
    print(json_data.keys())

  for bm in json_data.keys():
    if bm not in benign_dict:
      benign_dict[bm]        = False
      benign_data[bm]        = []
    
    for each_sample in json_data[bm]:
      if benign_data[bm] == []:
        benign_data[bm] = np.array(each_sample) 
      else:
        benign_data[bm] = np.vstack((benign_data[bm], each_sample))
  
  data_file.close()

print("Benign data read")
#--------------------------------------------------------------------------------------------------------------------------------#

# Malware data
for files in malware_files:

  with open(files, 'r') as data_file:
    json_data = json.load(data_file)
    print(json_data.keys())

  for bm in json_data.keys():
    if bm not in malware_dict:
      malware_dict[bm]        = False
      malware_data[bm]        = []
    
    for each_sample in json_data[bm]:
      if malware_data[bm] == []:
        malware_data[bm] = np.array(each_sample) 
      else:
        malware_data[bm] = np.vstack((malware_data[bm], each_sample))
  
  data_file.close()

print("Malware data read")
#--------------------------------------------------------------------------------------------------------------------------------#

In [ ]:
print(list(benign_data.items())[2][0])
print(len(benign_data))
print(len(benign_data['notepad2']))
print(len(benign_data['notepad2'][0]))
print(benign_data['notepad2'][0])
print(benign_data['pickledict'][0])

In [ ]:
print(list(malware_data.items())[2][0])
print(len(malware_data))
print(len(malware_data['813e882a972defdf261de319d1957c6a']))      #https://www.virustotal.com/gui/file/71b5fc8806f3b66b2b54e063d343b09393f25140fa5eb4498e31a7472a107c71/details
print(len(malware_data['813e882a972defdf261de319d1957c6a'][0]))   
print(malware_data['813e882a972defdf261de319d1957c6a'][0])
print(malware_data['19a94ba578e83c4f11d13878e17c27d2'][0])        #https://www.virustotal.com/gui/file/48b38729f7bda05deb7c7e682b342f925c459eff8c9891252afad617dc2260f0/detection

Sampling of data

In [ ]:
# Ensure equal number of benign keys and malware keys
sample_size = min(len(benign_data.keys()), len(malware_data.keys()))

print(sample_size)
print("benign bm numbers "  + str(len(benign_dict.keys())))
print("malware bm numbers " + str(len(malware_dict.keys())))
print("Chosen Size " + str(sample_size))

print("Sample size chosen")
#--------------------------------------------------------------------------------------------------------------------------------#

# Randomly choosing which benign keys are chosen
selected_benign_samples = random.sample(benign_data.keys(), sample_size)
for bm in list(benign_data.keys()):
  if bm not in selected_benign_samples:
    del benign_data[bm]

print("Excess benign data removed")
#--------------------------------------------------------------------------------------------------------------------------------#

# Just for show, since sample_size = malware_data.keys() size
selected_malware_samples = random.sample(malware_data.keys(), sample_size)

for bm in list(malware_data.keys()):
  if bm not in selected_malware_samples:
    del malware_data[bm]

print("Excess malware data removed")
#--------------------------------------------------------------------------------------------------------------------------------#

# Changing types to list
for bm in benign_data:
  benign_data[bm] = benign_data[bm].tolist()

for bm in malware_data:
  malware_data[bm] = malware_data[bm].tolist()

print("Type change to list done")
#--------------------------------------------------------------------------------------------------------------------------------#

Dump of data chosen for experiment (optional)

In [ ]:
'''
with open("/content/test1", 'w') as outfile:
  json.dump(benign_data, outfile, indent=2)
outfile.close()

with open("/content/test2", 'w') as outfile:
  json.dump(malware_data, outfile, indent=2)
outfile.close()
'''

Creating relevant functions

In [ ]:
# For combining benign and malware data into appropriate X_train etc variables
def split_and_stack(data, X_train, y_train, X_test, y_test, type='benign'):

  #marking benign as 1, malware as 0
  for bm in data:
    if type == 'benign':
      labels = np.ones(len(data[bm]))
    else:
      labels = np.zeros(len(data[bm]))
            
    single_X_train, single_X_test, single_y_train, single_y_test = train_test_split(data[bm], labels, test_size = 0.1, random_state = seed)

    if len(X_train) == 0:
        X_train = np.array(single_X_train)
        y_train = np.array(single_y_train)
        X_test  = np.array(single_X_test)
        y_test  = np.array(single_y_test)

    else:
        X_train = np.vstack((X_train, single_X_train))
        y_train = np.concatenate((y_train, single_y_train))
        X_test  = np.vstack((X_test, single_X_test))
        y_test  = np.concatenate((y_test, single_y_test))

  print("split_and_stack success")

  return X_train, y_train, X_test, y_test

# For LSTM model creation
def create_LSTM_model():
  model = Sequential()

  model.add(LSTM(num_neurons, dropout = 0.2, return_sequences = True, input_shape = (X_train.shape[1], 1)))
  model.add(LSTM(num_neurons, dropout = 0.2, return_sequences = True))
  model.add(LSTM(num_neurons, dropout = 0.2))

  model.add(Dense(1, activation = 'sigmoid'))

  print("create_LSTM_model success")

  return model

# For MLP (multi-layer perceptron) creation
def create_MLP_model():
  model = Sequential()

  model.add(Dense(num_neurons))
  model.add(Dropout(0.2))
  
  model.add(Dense(num_neurons))
  model.add(Dropout(0.2))

  model.add(Dense(num_neurons))
  model.add(Dropout(0.2))

  model.add(Dense(1, activation = 'sigmoid'))

  print("create_MLP_model success")

  return model

Creating classifier functions

In [ ]:
# Random Forest Classifier
def create_RF_model():

  # Variables
  max_depth = 40      # for parameter_i in list(xrange(30, 40, 1))]
  min_samples_split = 12
  min_samples_leaf = 12
  n_estimators = 100
  max_features = None
  random_state = seed

  model = RandomForestClassifier(max_depth = max_depth, min_samples_split = min_samples_split,
                                 min_samples_leaf = min_samples_leaf, n_estimators = n_estimators,
                                 max_features = max_features, random_state = random_state)
  
  return model

# Decision Tree Classifier
def create_DT_model():

  # Variables
  max_depth = 40     # for parameter_i in list(xrange(35, 45, 1))]
  min_samples_split = 12
  min_samples_leaf = 12
  presort = True
  max_features = None
  random_state = seed

  model = DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split,
                                 min_samples_leaf = min_samples_leaf, presort = True,
                                 max_features = max_features, random_state = random_state)
  
  return model

# Gaussian Naive Bayes
def create_NB_model():

  # Variables
  priors = [0.5, 0.5]

  model = GaussianNB(priors=priors)
  
  return model

# AdaBoost Classifier
def create_AB_model():

  # Variables
  algorithm = 'SAMME.R'   # is default
  n_estimators = 500      # for parameter_i in list(xrange(300, 1300, 100))]
  random_state = seed
  
  model = AdaBoostClassifier(algorithm = algorithm, n_estimators = n_estimators, random_state = random_state)
  
  return model

Data preprocessing; Shuffling and splitting

In [ ]:
X_train    = []
X_test     = []
y_train    = []
y_test     = []

# Combining benign and malware data
X_train, y_train, X_test, y_test = split_and_stack(benign_data, X_train, y_train, X_test, y_test, type='benign')
X_train, y_train, X_test, y_test = split_and_stack(malware_data, X_train, y_train, X_test, y_test, type='malware')

# Test prints
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

# Rejoining data to prepare for shuffling
X_data  = np.vstack((X_train, X_test))
y_data  = np.concatenate((y_train, y_test))

# Test prints; confirm no lost data
print(len(X_data))
print(len(y_data))

# Shuffling
X_data, y_data = shuffle(X_data, y_data, random_state = seed)

# Test print; reflects successful shuffle
print(np.where(y_data==1))

In [ ]:
print(np.amax(X_data))
print(np.amin(X_data))
print(np.std(X_data))
print(np.mean(X_data))

Data scaling

In [ ]:
X_data = X_data / 100000000000000

In [ ]:
print(np.amax(X_data))
print(np.amin(X_data))
print(np.std(X_data))
print(np.mean(X_data))

Training, val, test sets split;

80-20 --> (60-20)-20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=seed)   #80-20
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=seed)    #80 -> 60-20

# Test print; confirm intended split
print(len(X_train))
print(len(y_train))
print(len(X_val))
print(len(y_val))
print(len(X_test))
print(len(y_test))

Setting variables for 10-fold cross validation (k-fold)

In [ ]:
num_folds = 10
X = X_train
Y = y_train

num_data = len(X_train)    
nf = num_data//num_folds
print("num_data = ", num_data)
print("nf = ", nf)

idx = np.arange(num_data)
np.random.shuffle(idx)
X, Y = X[idx], Y[idx]

cross_val_accuracy = []
mse_values = []
time_per_fold = []

Model training parameters

In [ ]:
num_neurons = 64
epochs = 100
batch_size = 32
learning_rate = 0.001

loss = tf.keras.losses.BinaryCrossentropy()
#optimizer = 'SGD'
optimizer = 'Adam'

k-fold Cross-validation

In [ ]:

for fold in range(num_folds):

  # Manual splitting into train and test sets
  start, end = fold*nf, (fold+1)*nf
  X_eval_test, y_eval_test = X[start:end], Y[start:end]
  X_eval_train  = np.append(X[:start], X[end:], axis=0)
  y_eval_train = np.append(Y[:start], Y[end:], axis=0)

  # Model creation
  #model = create_LSTM_model()
  model = create_MLP_model()

  model.compile(optimizer=optimizer,
                loss=loss,
                metrics=['mse','accuracy'])
  
  # Reshape data to fit expected LSTM input shape
  # For LSTM
  #X_eval_train = np.reshape(X_eval_train, (X_eval_train.shape[0], X_eval_train.shape[1], 1))
  #X_eval_test  = np.reshape(X_eval_test , (X_eval_test.shape[0] , X_eval_test.shape[1] , 1))
  
  # Start time recorded
  start = time.time()
  print("Time started")

  # Model training
  history = model.fit(X_eval_train, y_eval_train,
                      epochs=epochs,
                      verbose=0,
                      use_multiprocessing=False,
                      validation_data=(X_eval_test, y_eval_test),
                      batch_size=batch_size)     # [loss, accuracy, val_loss, val_accuracy]
  
  end = time.time()
  print("Time ended")

  plt.plot(history.history['val_mse'], label='val_mse')
  plt.xlabel('epochs')
  plt.ylabel('val_mse')
  plt.legend() 

  # summing the val_accuracy values across all 10 folds
  cross_val_accuracy.append(history.history['val_accuracy'][-1])
  mse_values.append(history.history['val_mse'][-1])
  time_per_fold.append(end-start)
  print(cross_val_accuracy)
  print(mse_values)
  print(time_per_fold)


Calculations

In [ ]:

# Standard deviation
print(np.std(cross_val_accuracy))

# Mean
print(np.mean(cross_val_accuracy))


Model fitting (Classifiers)

In [ ]:
'''
# Creating model objects
RF_model = create_RF_model()
DT_model = create_DT_model()
NB_model = create_NB_model()
AB_model = create_AB_model()

# Fitting to training data
RF_model.fit(X_train, y_train)
DT_model.fit(X_train, y_train)
NB_model.fit(X_train, y_train)
AB_model.fit(X_train, y_train)

# Predicting on test data
y_pred_RF = RF_model.predict(X_test)
y_pred_DT = DT_model.predict(X_test)
y_pred_NB = NB_model.predict(X_test)
y_pred_AB = AB_model.predict(X_test)
'''

Classifier results

In [ ]:
'''
# Random Forest results
print("RF results:")
print("RF Precision:",metrics.precision_score(y_test, y_pred_RF))
print("RF Recall:",metrics.recall_score(y_test, y_pred_RF))
print("RF F1:",metrics.f1_score(y_test, y_pred_RF))

# Decision Tree results
print("DT results:")
print("DT Precision:",metrics.precision_score(y_test, y_pred_DT))
print("DT Recall:",metrics.recall_score(y_test, y_pred_DT))
print("DT F1:",metrics.f1_score(y_test, y_pred_DT))

# Gaussian NB results
print("NB results:")
print("NB Precision:",metrics.precision_score(y_test, y_pred_NB))
print("NB Recall:",metrics.recall_score(y_test, y_pred_NB))
print("NB F1:",metrics.f1_score(y_test, y_pred_NB))

# AdaBoost results
print("AB results:")
print("AB Precision:",metrics.precision_score(y_test, y_pred_AB))
print("AB Recall:",metrics.recall_score(y_test, y_pred_AB))
print("AB F1:",metrics.f1_score(y_test, y_pred_AB))
'''

Ensemble method (Classifiers, max voting)

In [ ]:
'''
# Using predictions from RF, DT, AB
# NB omitted due to poor precision
# Max voting same result as averaging as there's only 3 sets - a max vote will have the same rounded average with binary values

max_voting_pred = np.array([])

for i in range(0, len(X_test)):
    max_voting_pred = np.append(max_voting_pred, statistics.mode([y_pred_RF[i], y_pred_DT[i], y_pred_AB[i]]))

print("Classifier Ensemble Max Voting Precision:",metrics.precision_score(y_test, max_voting_pred))
print("Classifier Ensemble Max Voting Recall:",metrics.recall_score(y_test, max_voting_pred))
print("Classifier Ensemble Max Voting F1:",metrics.f1_score(y_test, max_voting_pred))
'''

Ensemble method (Classifiers, blending)

In [ ]:
'''
# Reshaping the predictions
y_pred_RF = y_pred_RF.reshape(y_pred_RF.shape[0],1)
y_pred_DT = y_pred_DT.reshape(y_pred_DT.shape[0],1)
y_pred_AB = y_pred_AB.reshape(y_pred_AB.shape[0],1)

# Test print
print(y_pred_RF.shape)

# Stacking the predictions
X_stacked = y_pred_RF
X_stacked = dstack((X_stacked, y_pred_DT))
X_stacked = dstack((X_stacked, y_pred_DT))

# Test print
print(X_stacked.shape)

# Reshaping stack
X_stacked = X_stacked.reshape((X_stacked.shape[0], X_stacked.shape[1]*X_stacked.shape[2]))

# Test print
print(X_stacked.shape)


# Logistic Regression model creation,fitting,predicting
model = LogisticRegression()

model.fit(X_stacked, y_test)

y_stacked_test_pred = model.predict(X_stacked)

print("Classifier Ensemble Blending Precision:",metrics.precision_score(y_test, y_stacked_test_pred))
print("Classifier Ensemble Blending Recall:",metrics.recall_score(y_test, y_stacked_test_pred))
print("Classifier Ensemble Blending F1:",metrics.f1_score(y_test, y_stacked_test_pred))
'''

Model evaluation

In [ ]:
'''
test_result = model.evaluate(X_test, y_test)    # Default batch size is 32; https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate
print(test_result)
'''